<a href="https://colab.research.google.com/github/IvanFP26/MachineLearning/blob/main/Week7/XGBoost_RetailPriceOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Nama : Ivan Fernanda Prayoga
#NIM : 1103204035
#XGBoost Retail Price Optimization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
# Membaca dataset dan mengubahnya menjadi DataFrame
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/retail_price.csv')

In [5]:
# Menampilkan DataFrame
data

,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
0,bed1,bed_bath_table,01-05-2017,1,45.95,15.100000,45.950000,39,161,2,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,45.900000
1,bed1,bed_bath_table,01-06-2017,3,137.85,12.933333,45.950000,39,161,2,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,45.950000
2,bed1,bed_bath_table,01-07-2017,6,275.70,14.840000,45.950000,39,161,2,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,45.950000
3,bed1,bed_bath_table,01-08-2017,4,183.80,14.287500,45.950000,39,161,2,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,45.950000
4,bed1,bed_bath_table,01-09-2017,2,91.90,15.100000,45.950000,39,161,2,...,89.9,3.9,18.776522,163.398710,4.4,24.324687,45.95,4.0,15.100000,45.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,bed5,bed_bath_table,01-05-2017,1,215.00,8.760000,215.000000,56,162,5,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,214.950000
672,bed5,bed_bath_table,01-06-2017,10,2090.00,21.322000,209.000000,56,162,5,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,215.000000
673,bed5,bed_bath_table,01-07-2017,59,12095.00,22.195932,205.000000,56,162,5,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,209.000000
674,bed5,bed_bath_table,01-08-2017,52,10375.00,19.412885,199.509804,56,162,5,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,205.000000


In [6]:
# Mengecek Informasi pada Dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  676 non-null    object 
 1   product_category_name       676 non-null    object 
 2   month_year                  676 non-null    object 
 3   qty                         676 non-null    int64  
 4   total_price                 676 non-null    float64
 5   freight_price               676 non-null    float64
 6   unit_price                  676 non-null    float64
 7   product_name_lenght         676 non-null    int64  
 8   product_description_lenght  676 non-null    int64  
 9   product_photos_qty          676 non-null    int64  
 10  product_weight_g            676 non-null    int64  
 11  product_score               676 non-null    float64
 12  customers                   676 non-null    int64  
 13  weekday                     676 non

In [7]:
# Periksa jumlah data yang hilang (NaN) dalam setiap kolom
missing_data = data.isnull().sum()

# Tampilkan data yang hilang
print(missing_data)

product_id                    0
product_category_name         0
month_year                    0
qty                           0
total_price                   0
freight_price                 0
unit_price                    0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_score                 0
customers                     0
weekday                       0
weekend                       0
holiday                       0
month                         0
year                          0
s                             0
volume                        0
comp_1                        0
ps1                           0
fp1                           0
comp_2                        0
ps2                           0
fp2                           0
comp_3                        0
ps3                           0
fp3                           0
lag_price                     0
dtype: int64


In [8]:
# Pemrosesan Data
data = data.dropna()

In [9]:
# Fungsi untuk menampilkan bentuk (shape) dari suatu objek data
data.shape

(676, 30)

In [10]:
# Mengubah kolom 'Bedroom2' menjadi tipe data integer
data['total_price'] = data['total_price'].astype(int)

# Mengubah kolom 'Bathroom' menjadi tipe data integer
data['freight_price'] = data['freight_price'].astype(int)

# Mengubah kolom 'Car' menjadi tipe data integer
data['unit_price'] = data['unit_price'].astype(int)

In [11]:
# Select subset of predictors
cols_to_use = ['product_score', 'comp_1', 'ps1', 'fp1']
X = data[cols_to_use]

In [12]:
# Select target
y = data.qty

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42,test_size=0.3)

In [13]:
# Import library
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
# Prediksi menggunakan data validasi
predictions = my_model.predict(X_valid)
print("Mean Absolute Errors : " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Errors : 11.447344270098974


In [15]:
my_model_0 = XGBRegressor(n_estimators=500)
my_model_0.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
# Prediksi menggunakan data validasi
predictions_0 = my_model_0.predict(X_valid)
print("Mean Absolute Erros : " + str(mean_absolute_error(predictions_0,y_valid)))

Mean Absolute Erros : 11.775682586754485


In [17]:
my_model_1 = XGBRegressor(n_estimators=110)
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=110, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
# Tentukan range nilai n_estimators yang akan diuji
n_estimators_list = [100, 150, 200, 250, 300]

# Inisialisasi sebuah list untuk menyimpan hasil MAE
mae_results = []

# Loop melalui setiap nilai n_estimators
for n_estimators in n_estimators_list:
    # Inisialisasi model dengan n_estimators tertentu
    model_grid = XGBRegressor(n_estimators=n_estimators)

    # Fitting model menggunakan data training
    model_grid.fit(X_train, y_train)

    # Lakukan prediksi pada data validasi
    predictions = model_grid.predict(X_valid)

    # Hitung Mean Absolute Error
    mae = mean_absolute_error(predictions, y_valid)

    # Tampilkan nilai n_estimators dan MAE
    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")

    # Tambahkan nilai MAE ke dalam list
    mae_results.append(mae)

n_estimators: 100, Mean Absolute Error: 11.447344270098974
n_estimators: 150, Mean Absolute Error: 11.616658371877788
n_estimators: 200, Mean Absolute Error: 11.701321264646323
n_estimators: 250, Mean Absolute Error: 11.752012856781777
n_estimators: 300, Mean Absolute Error: 11.766479452402134


In [19]:
my_model_2 = XGBRegressor(n_estimators=100)

my_model_2.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

# Prediksi menggunakan data validasi
predictions_2 = my_model_2.predict(X_valid)

print("Mean Absolute Error: " + str(mean_absolute_error(predictions_2, y_valid)))


Mean Absolute Error: 9.84429793992066


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [20]:
my_model_1 = XGBRegressor(n_estimators=100)
my_model_1.fit(X_train, y_train)

# Prediksi menggunakan data validasi
predictions_1 = my_model_1.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_1, y_valid)))

Mean Absolute Error: 11.447344270098974


In [21]:
my_model_3 = XGBRegressor(n_estimators=100,learning_rate=0.57,n_jobs=4)
my_model_3.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.57, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [22]:
# Prediksi menggunakan data validasi
predictions_3 = my_model_3.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_3, y_valid)))

Mean Absolute Error: 11.827587563562863
